In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125716")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "binitcluster"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, normal
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        #"learning_rate": normal(10, 3),
        #"keep_probability": uniform(0.05, 0.1),
        #"batch_size": choice(16, 32, 64, 128)
        '--C': choice(0.1, 0.3, 0.5, 0.7, 1.0),
        '--max_iter': uniform(10,100)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1,
                                         evaluation_interval = 1,
                                         delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = Estimator(source_directory = '.', entry_script = 'train.py', compute_target = cpu_cluster_name)

# Define the primary metric goal
goal = PrimaryMetricGoal("MAXIMIZE")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = ps,
                                   primary_metric_name = "accuracy",
                                   primary_metric_goal = goal,
                                   max_total_runs = 100,
                                   max_concurrent_runs = 4,
                                   policy = policy,
                                   estimator = est)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(run).show()



In [ ]:
run.get_status()
run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric() 
joblib.dump(best_run, 'model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=url)


In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
#from azureml.core.dataset import Dataset
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
ds = x.join(y)
#print(ds)
#print(y)
#ds = Dataset.from_pandas_dataframe(ds, path=None, in_memory=False)
train, test = train_test_split(ds, random_state=625, shuffle = True)
label = "y"
train.to_csv('train.csv')
#train = Dataset.from_pandas_dataframe(train, path=None, in_memory=True)
train = TabularDatasetFactory.from_delimited_files(path=url)
#print (train)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train,
    label_column_name="y",
    n_cross_validations=5,
    compute_target = cpu_cluster_name)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(ws, "automl_test_experiment")
run = experiment.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) #To print
joblib.dump(fitted_automl_model, "fitted_automl_model.joblib")
